# The Bessel-Gauss pulses

This notebook shows the electric fields for the Bessel-Gauss input profiles.

## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
from contextlib import ExitStack
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

## Load data

Here we select the input data. We select a list of input files, which will be then plotted.

In [ ]:
demos_path = os.path.join(os.environ['MULTISCALE_DEMOS'],'Bessel') # path to the data
series_version = 'v4'                                              # smulation series number

h5files = [os.path.join(demos_path, series_version, foo) for foo in 
            ['results_Bessel_1.h5', 'results_Bessel_7.h5', 'results_Bessel_3.h5', 'results_Bessel_9.h5']] # list of loaded simulations


# load the data
Nsim = len(h5files)
CUPRAD_res = []
with ExitStack() as stack:
    # Open all files and store file objects in a list
    files = [stack.enter_context(h5py.File(h5file, 'r')) for h5file in h5files]
    
    # Load data from each file and append it to CUPRAD_res
    for f in files:
        CUPRAD_res.append(dfC.get_data(f))


## Plot the propagating pulse
We plot the data for all the loaded simulations. We first set the shared limits for the plots defined relatively to the pulse duration of the first simulation and set the stride in $z$ for the plotting.

In [ ]:
tlim = np.asarray([t_plot_span*CUPRAD_res[0].pulse_duration_entry for t_plot_span in (-2. , 2.)])
frame_multiplier = 1 # stride in z for plotting

# rlim = 600



In [ ]:
# Code to generate the animated figure

k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res[0].tgrid,1.05*tlim)
# k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)


Nrows = Nsim//2 if (Nsim%2==0) else (Nsim//2)+1

fig, axes = plt.subplots(Nrows, 2, figsize=(15, Nrows*4))  # Create a 2x2 grid of subplots
axes = axes.flatten()


pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:len(CUPRAD_res)]):
    pc = ax.pcolormesh(1e15*CUPRAD_res[k1].tgrid[k_t_min:k_t_max],
                       1e6*CUPRAD_res[k1].rgrid,
                       CUPRAD_res[k1].E_zrt[0, :, k_t_min:k_t_max],
                       shading='auto', cmap='seismic')
    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax)
    cbar.ax.set_ylabel(r'$\mathcal{E}$ [V/m]', rotation=90)
    cbars.append(cbar)
    

    # Set axis properties
    ax.set_xlim(tlim)
    ax.set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[0]))
    ax.set_xlabel(r'$t~[\mathrm{fs}]$')

for k1 in range(Nrows): axes[2*k1].set_ylabel(r'$\rho~[\mu\mathrm{m}]$')  # Set ylabel only for the left plots
for k1 in range(Nsim,2*Nrows): axes[k1].axis('off')

# Create colorbars using list comprehensions


def update(frame):
    for k1 in range(len(CUPRAD_res)):
        # Update the data for each subplot
        data = CUPRAD_res[k1].E_zrt[frame_multiplier*frame, :, k_t_min:k_t_max]
        
        # Update the colors
        pcs[k1].set_array(data.ravel())
        max_value_symmetric = np.max(np.abs((data.min(), data.max())))
        pcs[k1].set_clim(-max_value_symmetric,max_value_symmetric)
        cbars[k1].update_normal(pcs[k1])

        axes[k1].set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[frame_multiplier*frame]))

    return pcs

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 

ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(CUPRAD_res[0].zgrid)//frame_multiplier, blit=True)

# Define the writer using ffmpeg for mp4 format and save it
ani_outpath = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'Bessel', 'export')
if not(os.path.exists(ani_outpath)): os.makedirs(ani_outpath)

print([foo.Nz for foo in CUPRAD_res])
Writer = matplotlib.animation.writers['ffmpeg']
writer = Writer(fps=5, metadata=dict(artist='Jan Vabek'), bitrate=1800)

ani.save(os.path.join(ani_outpath,'Bessel_pulse.mp4'), writer=writer)

HTML(ani.to_jshtml())
